In [1]:
%gui qt5
from chdrft.config.env import g_env
g_env.set_qt5(1)
init_jupyter()
import georinex as gr
#import ppp_tools.gpstime
import chdrft.utils.misc as cmisc
from chdrft.display.vtk import vtk_main_obj
import chdrft.display.vtk as opa_vtk
import  scipy.constants as constants
import chdrft.utils.K as K
from chdrft.geo.satsim import TileGetter
import mercantile
import pymap3d
from geopy.geocoders import Nominatim
import cv2
import calcephpy
from astroquery.jplhorizons import Horizons

from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris, EarthLocation
from astropy.coordinates import get_body_barycentric, get_body, get_moon
import datetime
from astropy import units as U
import math
from astropy.coordinates import SkyCoord
from astropy.coordinates import ICRS

W1023 23:57:04.611897 49451 __init__.py:121] Arrangement computation is unavailable


Got exception No module named 'unicorn'
Runnign magic qt5
done
False


In [2]:
tg = TileGetter()


def ecef_to_latlng(x,y,z):
  lng = math.atan2(y,x)
  lat = math.atan2(z, np.linalg.norm((x,y)))
  return Z.rad2deg(np.array((lat, lng)))

def eph_to_xyz(eph, unit=U.AU):
  a = SkyCoord(ra=eph['RA'], dec=eph['DEC'], distance=eph['delta'])
  return a.cartesian.xyz.to(unit).value.T

In [3]:
SkyCoord(ra=10.68458*U.degree, dec=41.26917*U.degree, distance=770*U.km).cartesian

<CartesianRepresentation (x, y, z) in km
    (568.71286542, 107.3008974, 507.88994292)>

In [4]:
x = Horizons(id=calcephpy.NaifId.MOON,
                  location=f'@{calcephpy.NaifId.EARTH}',
                  epochs={'start':'2019-01-01',
                          'stop':'2019-01-25',
                          'step': '5h'},
                  id_type='majorbody')
    

In [26]:
x.ephemerides()

targetname,datetime_str,datetime_jd,solar_presence,flags,RA,DEC,RA_app,DEC_app,RA_rate,DEC_rate,AZ,EL,AZ_rate,EL_rate,sat_X,sat_Y,sat_PANG,siderealtime,airmass,magextinct,V,surfbright,illumination,illum_defect,sat_sep,sat_vis,ang_width,PDObsLon,PDObsLat,PDSunLon,PDSunLat,SubSol_ang,SubSol_dist,NPole_ang,NPole_dist,EclLon,EclLat,r,r_rate,delta,delta_rate,lighttime,vel_sun,vel_obs,elong,elongFlag,alpha,lunar_elong,lunar_illum,sat_alpha,sunTargetPA,velocityPA,OrbPlaneAng,constellation,TDB-UT,ObsEclLon,ObsEclLat,NPole_RA,NPole_DEC,GlxLon,GlxLat,solartime,earth_lighttime,RA_3sigma,DEC_3sigma,SMAA_3sigma,SMIA_3sigma,Theta_3sigma,Area_3sigma,RSS_3sigma,r_3sigma,r_rate_3sigma,SBand_3sigma,XBand_3sigma,DoppDelay_3sigma,true_anom,hour_angle,alpha_true,PABLon,PABLat
---,---,d,---,---,deg,deg,deg,deg,arcsec / h,arcsec / h,deg,deg,arcsec / min,arcsec / min,arcsec,arcsec,deg,---,---,mag,mag,mag / arcsec2,%,arcsec,arcsec,---,arcsec,deg,deg,deg,deg,deg,arcsec,deg,arcsec,deg,deg,AU,km / s,AU,km / s,min,km / s,km / s,deg,---,deg,deg,%,deg,deg,deg,deg,---,s,deg,deg,deg,deg,deg,deg,---,min,arcsec,arcsec,arcsec,arcsec,deg,arcsec2,arcsec,km,km / s,Hz,Hz,s,deg,---,deg,deg,deg
str10,str17,float64,str1,str1,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str2,float64,float64,float64,int64,float64,float64,float64,str3,float64,float64,float64,float64,float64,float64,float64,int64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,float64
Moon (301),2019-Jan-01 00:00,2458484.5,,,221.25189,-10.62807,221.50148,-10.70481,1839.529,-626.439,--,--,--,--,--,--,--,--,999,--,-8.696,5.813,23.61701,1414.452,--,--,1851.789,6.498257,-6.560259,244.471768,0.418782,110.58,-786.51,16.9815,-919.832,100.1285,0.0107,0.981943908626,-0.8333253,0.00258725700416,0.0465195,0.02151756,29.8091732,1.0139735,58.0274,/L,121.8444,0.0,23.617,--,290.65,98.807,-5.11706,Lib,69.183918,222.3702079,5.0790448,266.6334,65.84147,342.8566,43.269527,--,0.0,--,--,--,--,--,--,--,--,--,--,--,--,--,--,121.8475,160.9139,5.2394
Moon (301),2019-Jan-01 05:00,2458484.708333333,,,223.85278,-11.48862,224.10396,-11.56207,1837.79,-607.934,--,--,--,--,--,--,--,--,999,--,-8.552,5.865,21.80065,1444.988,--,--,1847.826,6.462811,-6.46799,241.935675,0.413459,110.11,-762.95,16.1709,-918.032,100.3376,0.0105,0.981845073159,-0.8092927,0.00259280473259,0.0456981,0.0215637,29.771712,1.0121727,55.5464,/L,124.3287,0.0,21.8006,--,290.179,98.969,-5.05144,Lib,69.183925,225.0732426,5.0073389,266.63101,65.84133,344.935318,41.05997,--,0.0,--,--,--,--,--,--,--,--,--,--,--,--,--,--,124.332,162.3652,5.377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Moon (301),2019-Jan-24 13:00,2458508.041666667,,,173.07384,7.66832,173.31952,7.56251,2063.262,-760.616,--,--,--,--,--,--,--,--,999,--,-11.638,4.387,84.30285,309.1628,--,--,1969.547,4.670043,-5.54069,318.212017,-0.234724,109.29,716.47,24.2126,-980.173,124.0843,0.0084,0.985995388804,-0.5072888,0.00243256657769,0.0516912,0.02023104,31.0232689,1.078916,133.212,/L,46.6851,0.0,84.3029,--,289.277,297.536,-4.20708,Leo,69.184596,170.8746062,4.2946648,266.64333,65.86928,255.353312,62.975469,--,0.0,--,--,--,--,--,--,--,--,--,--,--,--,--,--,46.6812,147.3086,2.3418
Moon (301),2019-Jan-24 18:00,2458508.25,,,175.95068,6.60629,176.19532,6.5001,2047.014,-768.964,--,--,--,--,--,--,--,--,999,--,-11.553,4.441,82.46524,344.4523,--,--,1964.397,4.978799,-5.74518,315.683704,-0.242196,109.66,746.99,24.2637,-977.265,124.301,0.0088,0

In [5]:
eph_to_xyz(dict(RA=1 * U.deg, DEC=0 * U.deg, delta=123 * U.km))

array([0.000, 0.000, 0.000])

In [ ]:
app.exit_jup()

In [6]:
nav = gr.load('../../../dsp/gps/brdc3640.15n')
nav = gr.load('/home/benoit/Downloads/brdc1850.19n')

FileNotFoundError: /home/benoit/Downloads/brdc1850.19n

In [ ]:
nav

In [7]:
import datetime
import time
from astropy import constants as const

from astropy.coordinates.earth import OMEGA_EARTH
class Ephemerid:
  def __init__(self, s):
    self.s = s
    self.a = s['sqrtA'] ** 2
    
  @property
  def utc(self):
    time_utc= ppp_tools.gpstime.UTCFromGps(self.s['GPSWeek'], self.s['Toe'])
    time_utc = list(time_utc)
    time_utc[-1] = int(time_utc[-1])
    return datetime.datetime(*time_utc)

  
  def get_pos(self, t):
    
    dt = t -  self.utc
    dt_sec =dt.total_seconds()
    print('DT >>> ', dt)
    
    sv =self.s
    
    n0 = np.sqrt(const.GM_earth.value/self.a**3)  # computed mean motion
#    T = 2*pi / n0  # Satellite orbital period
    omega_e = OMEGA_EARTH.value

    n = n0 + sv['DeltaN']
    e = sv['Eccentricity']
# %% Kepler's eqn of eccentric anomaly
    Mk = sv['M0'] + n*dt_sec  # Mean Anomaly
    Ek = Mk + e * np.sin(Mk)  # Eccentric anomaly
# %% true anomaly
    nuK = np.arctan2(np.sqrt(1 - e**2) * np.sin(Ek),
                     np.cos(Ek) - e)
# %% latitude
    PhiK = nuK + sv['omega'] # argument of latitude
    duk = sv['Cuc'] * np.cos(2*PhiK) + sv['Cus']*np.sin(2*PhiK)  # argument of latitude correction
    uk = PhiK + duk  # corred argument of latitude
# %% inclination (same)
    dik = sv['Cic']*np.cos(2*PhiK) + sv['Cis']*np.sin(2*PhiK)  # inclination correction
    ik = sv['Io'] + sv['IDOT']*dt_sec + dik  # corrected inclination
# %% radial distance (same)
    drk = sv['Crc'] * np.cos(2*PhiK) + sv['Crs'] * np.sin(2*PhiK)  # radial correction
    rk = self.a * (1 - e * np.cos(Ek)) + drk  # corrected radial distance
# %% right ascension  (same)
    OmegaK = sv['Omega0'] + (sv['OmegaDot'] - omega_e)*dt_sec - omega_e*sv['Toe']
# %% transform
    Xk1 = rk * np.cos(uk)
    Yk1 = rk * np.sin(uk)

    X = Xk1 * np.cos(OmegaK) - Yk1 * np.sin(OmegaK) * np.cos(ik)

    Y = Xk1*np.sin(OmegaK) + Yk1 * np.cos(OmegaK) * np.cos(ik)

    Z = Yk1*np.sin(ik)
    return X,Y,Z


In [8]:
def analyse_sat(sat):
  tmp = nav.sel(sv=sat)
  df = tmp.to_dataframe().dropna()
  last = df.iloc[-1]
  a = Ephemerid(last)

  now = datetime.datetime(2015, 12, 30, 22)
  now = datetime.datetime.utcnow()
  xyz= a.get_pos(now)
  print(xyz)
  res = pymap3d.ecef2geodetic(*xyz)
  print(res)
  return res

In [ ]:
for val in nav['sv'].values:
  print('\n\n>>>', val)
  analyse_sat(val)

In [ ]:
gr.keplerian2ecef(tmp)

In [9]:
body_a = calcephpy.NaifId.EARTH
from astroquery.jplhorizons import Horizons
obj = Horizons(id='2019 OK', location=f'@{calcephpy.NaifId.EARTH}',
epochs={'start':'2019-06-01', 'stop':'2019-08-01',
'step':'1d'})
eph = obj.ephemerides()

In [ ]:
print(eph)

In [ ]:

start = '1969-07-16 16:40'
end = '1969-07-28'
start = '2019-06-01'
end = '2019-08-01'
body_center = calcephpy.NaifId.MARS
body_a = calcephpy.NaifId.PHOBOS
body_b = calcephpy.NaifId.DEIMOS
body_b =-399110
body_a = calcephpy.NaifId.MOON

bodies = [calcephpy.NaifId.DEIMOS, calcephpy.NaifId.PHOBOS, -3, -41, -74, ]
body_center = calcephpy.NaifId.MARS
bodies = [calcephpy.NaifId.EARTH, calcephpy.NaifId.MOON, '2019 OK']
body_center = calcephpy.NaifId.SUN
bodies = [calcephpy.NaifId.MOON, '2019 OK']
body_center = calcephpy.NaifId.EARTH
step = '1d'
objs = []
for body in bodies:
  objs.append(Horizons(id=body,
                 location=f'@{body_center}',
                 epochs={'start':start,
                         'stop':end,
                         'step':step},
                 id_type='majorbody'))


NameError: name 'U' is not defined

In [ ]:
eph_to_xyz(obj.ephemerides(), U.km)

In [ ]:
import chdrft.utils.K as K

In [ ]:
%gui qt4

In [ ]:
main = vtkMain()


for obj in objs:
  pts = eph_to_xyz(obj.ephemerides())
  actor = opa_vtk.create_line_actor(pts)
  main.ren.AddActor(actor)

sp = opa_vtk.SphereActor(1e-5, 10, (0,0,0), K.vispy_utils.Color('y').rgb)
main.ren.AddActor(sp)

In [ ]:
main.run()

In [ ]:
main.app.quit()

In [ ]:
from sbpy.data import Orbit
from astropy.time import Time
epoch = Time('2018-05-14', scale='utc')
eph = Orbit.from_horizons('Ceres', epochs=epoch)

In [ ]:
epoch2 = Time('2018-05-15', scale='utc')
eph2 = eph.oo_propagate(epoch2)

In [ ]:

import pyoorb as oo
oo.pyoorb.oorb_init()
from sbpy.data import Orbit
from astropy.time import Time
epoch = Time.now().jd + 100
ceres = Orbit.from_horizons('Ceres')      # doctest: +REMOTE_DATA
future_ceres = ceres.oo_propagate(epoch)  # doctest: +SKIP
print(future_ceres)  # doctest: +SKIP

In [ ]:
cart = eph2.oo_transform('CART')

In [ ]:
cart['x']

In [ ]:
for i in range(10):
  print(a[i].cartesian.x)

In [ ]:
type(elems[0])

In [ ]:
import reverse_geocoder as rg
coordinates = (51.5214588,-0.1729636),(9.936033, 76.259952),(37.38605,-122.08385)

results = rg.search(coordinates) # default mode = 2

print(results)

In [ ]:
gl = Nominatim(user_agent='VGYV7gGlNoWapA==')
obj = gl.geocode('Paris')
tile = mercantile.tile(obj.longitude, obj.latitude, 12)

In [ ]:
from chdrft.utils.cache import global_cache_list
global_cache_list

In [ ]:
tx = tg.get_tile(*tile)
img = K.opa_vtk.read_img_from_buf(tx)

In [ ]:
K.vispy_utils.render_for_meshes([cmisc.Attr(images=[img[::-1]])])
#K.plot_img(img[::-1])

In [ ]:
open('/tmp/test.png', 'wb').write(tx)

In [ ]:
img = cv2.imread('/tmp/test.png', 1)

In [ ]:
print(img.shape)

In [ ]:
#img_vtk = K.opa_vtk.numpy_to_vtk_image(np.ascontiguousarray(img[:,:,0]))
img_vtk = K.opa_vtk.numpy_to_vtk_image(np.ascontiguousarray(img.reshape((1,) + img.shape)))
tex = K.opa_vtk.reader2tex(img_vtk)
#tex = K.opa_vtk.jpeg2tex('/tmp/test.png')

actor= K.opa_vtk.TriangleActor(tex=tex).full_quad(Z.opa_struct.g_unit_box.poly(z_coord=1)).build()

In [ ]:
main = vtkMain()
#$main.ren.AddActor(actor)
main.run()

In [16]:
  
class Quad:
  
  def __init__(self, box, depth):
    self.box = box
    self.depth = depth
    self._children = None
    
  @property
  def children(self):
    if self._children is None:
      self._children = []
      for qd in self.box.quadrants:
        self._children.append(Quad(qd, self.depth+1))
    return self._children
  
  def __iter__(self):
    return iter(self.children)
  
  
  @staticmethod
  def Root():
    return Quad(Z.opa_struct.g_unit_box, 0)    
 

In [17]:
class Consts:
  EARTH_ELLIPSOID = pymap3d.Ellipsoid('wgs84')
  MOON_ELLIPSOID = pymap3d.Ellipsoid('moon')
 
class TMSQuad:
  LMAX=85.05113
  MAX_DEPTH=20
  
  def __init__(self, x, y, z):
    self.x = x
    self.y = y
    self.z = z
    self._children = None
    
  @property
  def children(self):
    if self._children is None:
      self._children = []
      if self.z +1 < TMSQuad.MAX_DEPTH:
        for i in range(4):
          self._children.append(TMSQuad(2*self.x+(i&1), 2*self.y+(i>>1), self.z+1))
    return self._children
  
  def __iter__(self):
    return iter(self.children)
  
  @property
  def box_latlng(self):
    bounds = mercantile.bounds(*self.xyz)
    box = Z.Box(low=(bounds.west, bounds.south), high=(bounds.east, bounds.north))
    return box
    
  @property
  def quad_ecef(self):
    p = self.box_latlng.poly()
    return Z.opa_struct.Quad(np.stack(pymap3d.geodetic2ecef(p[:,1], p[:,0], 0, ell=Consts.EARTH_ELLIPSOID), axis=-1) / 1e3) 
  
  @property
  def xyz(self):
    return self.x, self.y, self.z
  
  def tile(self, tg):
    return tg.get_tile(*self.xyz)
  
  @staticmethod
  def Root():
    return TMSQuad(0,0,0)    
  
  
def do_visit(obj, func):
  if func(obj):
    for x in obj:
      do_visit(x, func)

In [18]:
class SimpleVisitor:
  def __init__(self, max_depth=2):
    self.max_depth = max_depth
    self.actors = []
    self.pts = []
    
  def __call__(self, obj):
    if obj.z < self.max_depth: return 1
    tx = obj.tile(tg)
    img = K.opa_vtk.read_img_from_buf(tx)
    actor = opa_vtk.TriangleActor(tex=K.opa_vtk.numpy2tex(img)).full_quad_yinv(obj.quad_ecef).build()
    self.actors.append(actor)
    self.pts.extend(obj.quad_ecef.pts)
    return 0
  
  def run_tms(self):
    root = TMSQuad.Root()
    do_visit(root, self)

In [19]:
class CylindricalVisitor:
  def __init__(self, max_depth=2):
    self.max_depth = max_depth
    self.ta = opa_vtk.TriangleActor()
    self.real_box = Z.Box(low=(-np.pi, -np.pi/2), high= (np.pi, np.pi/2))
    self.pts = []
    
  def __call__(self, obj):
    if obj.depth < self.max_depth: return 1
    base_p = obj.box.poly()
    p = self.real_box.from_box_space(base_p)
    
    pmap = np.stack(pymap3d.geodetic2ecef(p[:,1], p[:,0], 0, ell=Consts.MOON_ELLIPSOID, deg=0), axis=-1) / 1e3
    self.pts.extend(pmap)
    self.ta.add_quad(pmap, base_p)
    return 0
  
  def run(self):
    root = Quad.Root()
    do_visit(root, self)
    
def create_moon_data():
  tex = K.opa_vtk.jpeg2tex('/home/benoit/Downloads/Moon_LRO_LOLA_global_LDEM_1024.jpg')
  width, height =  tex.GetInput().GetDimensions()[:2]
  cv = CylindricalVisitor(4)
  cv.run()
  actor = cv.ta.set_tex(tex).build()
  return cmisc.Attr(actor=actor, pts=cv.pts)
  

In [20]:
def make_norm(a):
  return a / np.linalg.norm(a)
def make_orth_norm(a, b):
  a = np.array(a)
  b = make_norm(np.array(b))
  return make_norm(a - np.dot(a, b) * b)

def compute_angles(center, focal_point, up, pts):
  center, focal_point, up, pts = cmisc.to_numpy_args(center, focal_point, up, pts)
  pts =pts.reshape((-1, 3))
  
  dpt = pts - focal_point
  dfocal = focal_point - center
  dfocal_len = np.linalg.norm(dfocal)
  z = dfocal / dfocal_len
  
  y = make_orth_norm(up, z)
  x = make_norm(np.cross(y, z))
  pt_len = np.linalg.norm(dpt)
  dpc = pts - center
  dproj = np.dot(dpc, z)
  
  #xp = np.arcsin(np.dot(pts, x) / np.linalg.norm(pts-center, axis=1))
  #yp = np.arcsin(np.dot(dpt, y) / np.linalg.norm(pts-center, axis=1))
  
  xp = np.arctan2(np.dot(dpc, x), dproj)
  yp = np.arctan2(np.dot(dpc, y), dproj)
  
  res = np.stack((xp, yp), axis=-1)
  return res,y


def compute_cam_parameters(center, focal_point, up, pts, expand=0.05, aspect=None):
  angles,y = compute_angles(center, focal_point, up, pts)
  box = Z.Box.FromPoints(angles)
  print(box)
  box = box.center_on(np.zeros((2,)))
  print(box)
  box =box.expand(1 + expand)
  if aspect is not None:
    box = box.set_aspect(aspect)
  return cmisc.Attr(box=box, y=y)
  
#res =compute_cam_parameters((0, 0, 0), (0,0, 1), (0, 1, 0), [(1, 1, 1), (1,1,2)], expand=0.05, aspect=1.5)

print(np.min(pts_list, axis=0))
print(np.max(pts_list, axis=0))
print(focal_point)
params =compute_cam_parameters(mars_pos, focal_point, up, pts_list, expand=0.10, aspect=main.aspect)
print(params.box)
print(params.box.width)
print(params.box.height)
print(main.aspect)
print(params.box.aspect)

NameError: name 'pts_list' is not defined

In [ ]:
moon_data = create_moon_data()
main = vtkMain()
main.ren.AddActor(moon_data.actor)
main.run()
  

In [ ]:

t = Time(datetime.datetime.utcnow()-datetime.timedelta(days=0))
with solar_system_ephemeris.set('builtin'):
  moon = get_body('moon', t, None) 
  mars = get_body('mars', t, None) 
  earth = get_body('earth', t, None) 
  moon = moon.transform_to('hcrs')
  mars = mars.transform_to('hcrs')
  earth = earth.transform_to('hcrs')
  moon_pos = moon.cartesian.xyz.to(u.km).value
  mars_pos = mars.cartesian.xyz.to(u.km).value
  earth_pos = earth.cartesian.xyz.to(u.km).value
  up = (0,0,1)
  
  main = vtkMain()
  
  moon_data = create_moon_data()
  moon_actor = moon_data.actor
  moon_actor.GetProperty().SetAmbient(1)
  moon_actor.GetProperty().SetDiffuse(0)
  moon_actor.SetPosition(*moon_pos)

  u = SimpleVisitor(4)
  u.run_tms()
  earth_assembly = K.opa_vtk.vtk.vtkAssembly()
  for actor in u.actors:
    earth_assembly.AddPart(actor)
    actor.GetProperty().SetAmbient(1)
    actor.GetProperty().SetDiffuse(0)
  earth_assembly.SetPosition(*earth_pos)
  
  
  pts_list = np.concatenate((np.array(u.pts) +earth_pos, np.array(moon_data.pts) + moon_pos))
  focal_point = np.mean(pts_list, axis=0)
  #pts_list = np.concatenate((np.array(moon_data.pts) + moon_pos, ))
  #focal_point = moon_pos
  #focal_point = earth_pos + (moon_pos - earth_pos)*0.59

  main.ren.AddActor(earth_assembly)
  main.ren.AddActor(moon_actor)
    
  
  params =compute_cam_parameters(mars_pos, focal_point, up, pts_list, expand=0.05, aspect=main.aspect)
  
  #main.cam.SetFocalPoint(*moon_pos)
  main.cam.SetClippingRange(1, 1e20)
  angle_y = Z.rad2deg(params.box.height)
  print('ANGLE > > ', angle_y, params.box.height)
  
  main.cam.SetPosition(*mars_pos)
  main.cam.SetFocalPoint(*focal_point)
  main.cam.SetViewAngle(angle_y)
  main.cam.SetViewUp(*params.y)

  
  main.run(reset=0)


In [ ]:

start = '2019-07-24'
end = '2019-07-28'
bodies = [calcephpy.NaifId.MOON, '2019 OK']
starts = {}
starts[calcephpy.NaifId.MOON] = '2019-07-01'
body_center = calcephpy.NaifId.EARTH
step = '30m'
steps = {}
steps[calcephpy.NaifId.MOON] = '5h'
objs = []
for body in bodies:
  objs.append(Horizons(id=body,
                 location=f'@{body_center}',
                 epochs={'start':starts.get(body, start),
                         'stop':end,
                         'step':steps.get(body, step)},
                 id_type='majorbody'))
main = vtkMain()

for obj in objs:
  eph = obj.ephemerides()
  pts = eph_to_xyz(eph, U.km)
  dpts = np.diff(pts, axis=0)
  speed = np.linalg.norm(dpts, axis=1)
  dists = np.linalg.norm(pts, axis=1)
  imin=  np.argmin(dists)
  
  print(obj, eph['datetime_str'][imin], speed[imin], dists[imin-1:imin+2])
  print(min(speed), max(speed))
  actor = opa_vtk.create_line_actor(pts)
  main.ren.AddActor(actor)

assert 0
#  JPLHorizons instance "301"; location=@399, epochs={'start': '2019-07-01', 'stop': '2019-07-28', 'step': '5h'}, id_type=majorbody 2019-Jul-05 04:00 19468.309901514996 [363785.679 363745.979 363764.904]
#JPLHorizons instance "2019 OK"; location=@399, epochs={'start': '2019-07-24', 'stop': '2019-07-28', 'step': '1m'}, id_type=majorbody 2019-Jul-25 01:21 1472.0863310984244 [71370.228 71349.445 71358.746]
u = SimpleVisitor(4)
u.run_tms()
earth_assembly = K.opa_vtk.vtk.vtkAssembly()
for actor in u.actors:
  earth_assembly.AddPart(actor)
  actor.GetProperty().SetAmbient(1)
  actor.GetProperty().SetDiffuse(0)
earth_pos = (0,0,0)
earth_assembly.SetPosition(*earth_pos)
  
main.ren.AddActor(earth_assembly)
main.run()

In [ ]:
print(eph)

In [11]:
start = '2015-10-12'
end = '2015-10-13'
step = '1d'
steps = {}
starts={}
bodies = [calcephpy.NaifId.MOON, calcephpy.NaifId.EARTH]
body_center = 'LRO'
objs = []
for body in bodies:
  obj = Horizons(id=body,
                 location=f'@{body_center}',
                 epochs={'start':starts.get(body, start),
                         'stop':end,
                         'step':steps.get(body, step)},
                 id_type='majorbody')
  
  print(obj.ephemerides())
  print(obj.uri)
  objs.append(obj)
assert 0
main = vtkMain()

for obj in objs:
  eph = obj.ephemerides()
  pts = eph_to_xyz(eph, U.km)
  dpts = np.diff(pts, axis=0)
  speed = np.linalg.norm(dpts, axis=1)
  dists = np.linalg.norm(pts, axis=1)
  imin=  np.argmin(dists)
  
  print(obj, eph['datetime_str'][imin], speed[imin], dists[imin-1:imin+2])
  print(min(speed), max(speed))
  actor = opa_vtk.create_line_actor(pts)
  main.ren.AddActor(actor)

assert 0
#  JPLHorizons instance "301"; location=@399, epochs={'start': '2019-07-01', 'stop': '2019-07-28', 'step': '5h'}, id_type=majorbody 2019-Jul-05 04:00 19468.309901514996 [363785.679 363745.979 363764.904]
#JPLHorizons instance "2019 OK"; location=@399, epochs={'start': '2019-07-24', 'stop': '2019-07-28', 'step': '1m'}, id_type=majorbody 2019-Jul-25 01:21 1472.0863310984244 [71370.228 71349.445 71358.746]
u = SimpleVisitor(4)
u.run_tms()
earth_assembly = K.opa_vtk.vtk.vtkAssembly()
for actor in u.actors:
  earth_assembly.AddPart(actor)
  actor.GetProperty().SetAmbient(1)
  actor.GetProperty().SetDiffuse(0)
earth_pos = (0,0,0)
earth_assembly.SetPosition(*earth_pos)
  
main.ren.AddActor(earth_assembly)
main.run()

targetname    datetime_str   datetime_jd ... alpha_true  PABLon   PABLat 
   ---            ---             d      ...    deg       deg      deg   
---------- ----------------- ----------- ... ---------- -------- --------
Moon (301) 2015-Oct-12 00:00   2457307.5 ...   120.9752  75.2879 -24.8133
Moon (301) 2015-Oct-13 00:00   2457308.5 ...    74.8747 359.0453 -32.2853
https://ssd.jpl.nasa.gov/horizons_batch.cgi?batch=1&TABLE_TYPE=OBSERVER&QUANTITIES=%271%2C2%2C3%2C4%2C5%2C6%2C7%2C8%2C9%2C10%2C11%2C12%2C13%2C14%2C15%2C16%2C17%2C18%2C19%2C20%2C21%2C22%2C23%2C24%2C25%2C26%2C27%2C28%2C29%2C30%2C31%2C32%2C33%2C34%2C35%2C36%2C37%2C38%2C39%2C40%2C41%2C42%2C43%27&COMMAND=%22301%22&SOLAR_ELONG=%220%2C180%22&LHA_CUTOFF=0&CSV_FORMAT=YES&CAL_FORMAT=BOTH&ANG_FORMAT=DEG&APPARENT=AIRLESS&REF_SYSTEM=J2000&CENTER=%27%40LRO%27&START_TIME=%222015-10-12%22&STOP_TIME=%222015-10-13%22&STEP_SIZE=%221d%22&SKIP_DAYLT=NO
 targetname    datetime_str   datetime_jd ... alpha_true  PABLon  PABLat
    ---            

AssertionError: 

In [14]:
print(obj.ephemerides().columns)

<TableColumns names=('targetname','datetime_str','datetime_jd','solar_presence','flags','RA','DEC','RA_app','DEC_app','RA_rate','DEC_rate','AZ','EL','AZ_rate','EL_rate','sat_X','sat_Y','sat_PANG','siderealtime','airmass','magextinct','V','surfbright','illumination','illum_defect','sat_sep','sat_vis','ang_width','PDObsLon','PDObsLat','PDSunLon','PDSunLat','SubSol_ang','SubSol_dist','NPole_ang','NPole_dist','EclLon','EclLat','r','r_rate','delta','delta_rate','lighttime','vel_sun','vel_obs','elong','elongFlag','alpha','IB_elong','IB_illum','sat_alpha','sunTargetPA','velocityPA','OrbPlaneAng','constellation','TDB-UT','ObsEclLon','ObsEclLat','NPole_RA','NPole_DEC','GlxLon','GlxLat','solartime','earth_lighttime','RA_3sigma','DEC_3sigma','SMAA_3sigma','SMIA_3sigma','Theta_3sigma','Area_3sigma','RSS_3sigma','r_3sigma','r_rate_3sigma','SBand_3sigma','XBand_3sigma','DoppDelay_3sigma','true_anom','hour_angle','alpha_true','PABLon','PABLat')>


In [ ]:
import time
import pymap3d
cam = main.ren.GetActiveCamera()
d = np.linalg.norm(cam.GetPosition())
def spherical_to_xyz(alpha, phi):
  return np.array((np.cos(alpha) * np.cos(phi), np.sin(alpha) * np.cos(phi), np.sin(phi)))
for t in np.linspace(0,1, 1000):
  pos = spherical_to_xyz(2*np.pi*3*t  / 2, 0*np.cos(2*np.pi*t) * np.pi /2 *0.4 ) * d
  cam.SetPosition(*pos)
  print(np.linalg.norm(pos))
  main.ren.ResetCameraClippingRange()
  main.ren_win.Render()
  time.sleep(0.01)

In [ ]:
print(obj.ephemerides()['datetime_str'][0])

In [ ]:
def mulnorm(m, v):
  v = list(v)+[1]
  v = m.MultiplyPoint(v)
  return np.array(v[:3])/v[-1]

z = make_norm(focal_point - mars_pos)
y = make_orth_norm((1,0,0), z)
m = main.cam.GetCompositeProjectionTransformMatrix(main.aspect, -1,1 )
print(Z.deg2rad(angle_y))
y = params.y
print(np.linalg.norm(np.cross(y,z)))
x = make_norm(np.cross(y,z))
for pt in pts_list:
  print(np.arcsin(np.dot(x, pt - mars_pos) / np.linalg.norm(pt - mars_pos)))
  print(mulnorm(m, pt))
  print()

In [ ]:
app.exit_jup()
